# Packages

In [50]:
import numpy as np

from ortools.graph.python import max_flow, min_cost_flow

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Route Optimization\\OR-Tools'

# Overview

Many problems in computer science can be represented by a graph consisting of nodes and links between them. Examples are **network flow** problems, which involve transporting goods or material across a network, such as a railway system.

You can represent a network flow by a graph whose nodes are cities and whose arcs are rail lines between them. (They're called **flows** because their properties are similar to those of water flowing through a network of pipes.)

A key constraint in network flows is that each arc has a **capacity** — the maximum amount that can be transported across the arc in a fixed period of time.

The **maximum flow problem** is to determine **the maximum total amount that can be transported across all arcs** in the network, subject to the capacity constraints.

The first person to study this problem was the Russian mathematician A.N. Tolstoi, in the 1930s. The map below shows the actual railway network for which he wanted to find a maximum flow.

<p align = center>
    <img src = https://developers.google.com/static/optimization/images/railway_network.png>
</p>

# Maximum Flows

The problem is defined by the following graph, which represents a transportation network:

<p align = center>
    <img src = https://developers.google.com/static/optimization/images/flow/max_flow.svg>
</p>

We wish to transport material from node 0 (the **source**) to node 4 (the **sink**). The numbers next to the arcs are their capacities — the capacity of an arc is the maximum amount that can be transported across it in a fixed period of time. The capacities are the constraints for the problem.

A **flow** is an assignment of a non-negative number to each arc (the **flow amount**) that satisfies the following **flow conservation** rule: At each node, other than the source or the sink, the total flow of all arcs leading in to the node equals the total flow of all arcs leading out of it.

The max flow problem is to find a flow for which the sum of the flow amounts for the entire network is as large as possible. The following sections present a programs to find the maximum flow from the source (0) to the sink (4).

In [51]:
# Declare solver, instantiate a SimpleMaxFlow solver
smf = max_flow.SimpleMaxFlow()

You define the graph for the problem with three arrays, for the start nodes, end nodes, and capacities of the arcs. The length of each array equals the number of arcs in the graph.

For each `i`, arc `i` goes from `start_nodes[i]` to `end_nodes[i]`, and its capacity is given by `capacities[i]`. The next section shows how to create the arcs using this data.

In [52]:
# Define three paralell arrays: start_nodes, end_nodes, and the capacities
# between each pair.
# For instance, the arc from node 0 to node 1 has a capacity of 20
start_nodes = np.array([0, 0, 0, 1, 1, 2, 2, 3, 3])
end_nodes = np.array([1, 2, 3, 2, 4, 3, 4, 2, 4])
capacities = np.array([20, 30, 10, 40, 30, 10, 20, 5, 20])

For each start node and end node, we create an arc from start node to end node with the given capacity, using the method `AddArcWithCapacity`. The capacities are the constraints for the problem.

In [53]:
# Add arcs in bulk
# We could have used add_arc_with_capacity(start, end, capacity)
all_arcs = smf.add_arcs_with_capacity(start_nodes, end_nodes, capacities)

Now that all the arcs have been defined, all that remains is to invoke the solver and display the results. We invoke the `Solve()` method, providing the source (0) and sink(4).

In [54]:
# Find the maximum flow between node 0 and node 4
status = smf.solve(0, 4)

if status != smf.OPTIMAL:
    print('There was an issue with the max flow input:')
    print(f'Status: {status}')
    exit(1)
print('Max flow:', smf.optimal_flow())
print()
print(' Arc    Flow / Capacity')
solution_flows = smf.flows(all_arcs)
for arc, flow, capacity in zip(all_arcs, solution_flows, capacities):
    print(f'{smf.tail(arc)} / {smf.head(arc)}  {flow:3}  / {capacity:3}')
print('Source side min-cut:', smf.get_source_side_min_cut())
print('Sink side min-cut:', smf.get_sink_side_min_cut())

Max flow: 60

 Arc    Flow / Capacity
0 / 1   20  /  20
0 / 2   30  /  30
0 / 3   10  /  10
1 / 2    0  /  40
1 / 4   20  /  30
2 / 3   10  /  10
2 / 4   20  /  20
3 / 2    0  /   5
3 / 4   20  /  20
Source side min-cut: [0]
Sink side min-cut: [4, 1]


# Minimum Cost Flows

Closely related to the max flow problem is the **minimum cost (min cost)** flow problem, in which each arc in the graph has a unit cost for transporting material across it. The problem is to find a flow with the least total cost.

The min cost flow problem also has special nodes, called supply nodes or demand nodes, which are similar to the source and sink in the max flow problem. Material is transported from supply nodes to demand nodes.
- At a **supply node**, a positive amount — the supply — is added to the flow. A supply could represent production at that node, for example.
- At a **demand node**, a negative amount — the demand — is taken away from the flow. A demand could represent consumption at that node, for example.

For convenience, we'll assume that all nodes, other than supply or demand nodes, have *zero supply (and demand)*.

For the min cost flow problem, we have the following flow conservation rule, which takes the supplies and demands into account: At each node, the *total flow leading out of the node minus the total flow leading in to the node* equals the supply (or demand) at that node.

The graph below shows a min cost flow problem. The arcs are labeled with pairs of numbers: the first number is the capacity and the second number is the cost. The numbers in parentheses next to the nodes represent supplies or demands. Node 0 is a supply node with supply 20, while nodes 3 and 4 are demand nodes, with demands -5 and -15, respectively.

<p align = center>
    <img src = https://developers.google.com/static/optimization/images/flow/min_cost_flow.svg>
</p>

In [55]:
# Instantiate a SimpleMinCostFlow solver
smcf = min_cost_flow.SimpleMinCostFlow()

The following code defines the data for the problem. In this case, there are four arrays for the start nodes, end nodes, capacities, and unit costs. Again, the length of the arrays is the number of arcs in the graph.

In [56]:
# Define four parallel arrays: sources, destinations, capacities, and unit costs between each pair.
# For instance, the arc from node 0 to node 1 has a capacity of 15.
start_nodes = np.array([0, 0, 1, 1, 1, 2, 2, 3, 4])
end_nodes = np.array([1, 2, 2, 3, 4, 3, 4, 4, 2])
capacities = np.array([15, 8, 20, 4, 10, 15, 4, 20, 5])
unit_costs = np.array([4, 4, 2, 2, 6, 1, 3, 2, 3])

# Define an array of supplies at each node.
supplies = [20, 0, 0, -5, -15]

For each start node and end node, we create an arc from start node to end node with the given capacity and unit cost, using the method `AddArcWithCapacityAndUnitCost`.

The solver's `SetNodeSupply` method creates a vector of supplies for the nodes.

In [57]:
# Add arcs, capacities, and costs in bulk using numpy
all_arcs = smcf.add_arcs_with_capacity_and_unit_cost(
    start_nodes, end_nodes, capacities, unit_costs
)

# Add supply for each nodes
smcf.set_nodes_supply(np.arange(0, len(supplies)), supplies)

# Find min cost flow
status = smcf.solve()

# Display results
if status != smcf.OPTIMAL:
    print('THere was an issue with the min cost flow input.')
    print('Status:', status)
    exit(1)
print(f'Minimum cost: {smcf.optimal_cost()}')
print()
print(' Arc    Flow / Capacity Cost')
solution_flows = smcf.flows(all_arcs)
costs = solution_flows * unit_costs
for arc, flow, cost in zip(all_arcs, solution_flows, costs):
    print(f'{smcf.tail(arc):1} -> {smcf.head(arc)}  {flow:3}  / {smcf.capacity(arc):3}')

Minimum cost: 150

 Arc    Flow / Capacity Cost
0 -> 1   12  /  15
0 -> 2    8  /   8
1 -> 2    8  /  20
1 -> 3    4  /   4
1 -> 4    0  /  10
2 -> 3   12  /  15
2 -> 4    4  /   4
3 -> 4   11  /  20
4 -> 2    0  /   5


# Assignment as Min Cost Flow Problem

You can use the min cost flow solver to solve special cases of the assignment problem. In fact, min cost flow can often return a solution faster than either the MIP or CP-SAT solver. However, MIP and CP-SAT can solve a larger class of problems than min cost flow, so in most cases MIP or CP-SAT are the best choices.

The following sections present Python programs that solve the following assignment problems using the min cost flow solver:
- A simple linear assignment example.
- An assignment problem with teams of workers.

## Linear assignment example

In [58]:
# Instantiate a SimpleMinCostFlow solver
smcf = min_cost_flow.SimpleMinCostFlow()

The flow diagram for the problem consists of the bipartite graph for the cost matrix, with a source and sink added.

<p align = center>
    <img src = https://developers.google.com/static/optimization/images/flow/assignment_min_cost_flow.svg>
</p>

Note: The numbering of the workers and tasks is slightly different than in the section Linear Assignment Solver, because the min cost flow solver requires all nodes in the graph to be numbered distinctly.

The data contains the following four arrays, corresponding to the start nodes, end nodes, capacities, and costs for the problem. The length of each array is the number of arcs in the graph.

In [59]:
# Define the directed graph for the flow.
start_nodes = [0, 0, 0, 0] + [
    1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4
] + [5, 6, 7, 8]
end_nodes = [1, 2, 3, 4] + [5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8
                           ] + [9, 9, 9, 9]
capacities = [1, 1, 1, 1] + [
    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
] + [1, 1, 1, 1]
costs = (
    [0, 0, 0, 0] +
    [90, 76, 75, 70, 35, 85, 55, 65, 125, 95, 90, 105, 45, 110, 95, 115] +
    [0, 0, 0, 0])

source = 0
sink = 9
tasks = 4
supplies = [tasks, 0, 0, 0, 0, 0, 0, 0, 0, -tasks]

To make clear how the data is set up, each array is divided into three sub-arrays:
- The first array corresponds to arcs leading out of the source.
- The second array corresponds to the arcs between workers and tasks. For the costs, this is just the cost matrix (used by the linear assignment solver), flattened into a vector.
- The third array corresponds to the arcs leading into the sink.

The data also includes the vector supplies, which gives the supply at each node.

**How does the min cost flow problem above represent an assignment problem?** First, since the capacity of every arc is 1, the supply of 4 at the source forces each of the four arcs leading into the workers to have a flow of 1.

Next, the flow-in-equals-flow-out condition forces the flow out of each worker to be 1. If possible, the solver would direct that flow across the minimum cost arc leading out of each worker. However, the solver cannot direct the flows from two different workers to a single task. If it did, there would be a combined flow of 2 at that task, which could not be sent across the single arc with capacity 1 from the task to the sink. This means that the solver can only assign a task to a single worker, as required by the assignment problem.

Finally, the flow-in-equals-flow-out condition forces each task to have an outflow of 1, so each task is performed by some worker.

In [60]:
# Create graph and constraints
# Add each arcs
for i in range(len(start_nodes)):
    smcf.add_arc_with_capacity_and_unit_cost(start_nodes[i], end_nodes[i],
                                            capacities[i], costs[i])

# Add node supplies
for i in range(len(supplies)):
    smcf.set_node_supply(i, supplies[i])

# Invoke solver, find minimum cost flow between node 0 and node 10
status = smcf.solve()

The solution consists of the arcs between workers and tasks that are assigned a flow of 1 by the solver. (Arcs connected to the source or sink are not part of the solution.)

The program checks each arc to see if it has flow 1, and if so, prints the `Tail` (start node) and the `Head` (end node) of the arc, which correspond to a worker and task in the assignment.

In [61]:
if status == smcf.OPTIMAL:
    print('Total cost =', smcf.optimal_cost())
    print()
    for arc in range(smcf.num_arcs()):
        # Can ignore arcs leading out of source or into sink
        if smcf.tail(arc) != source and smcf.head(arc) != sink:
            # Arcs in the solution have a flow value of 1
            # Their start and end nodes give an assignment of worker to task
            if smcf.flow(arc) > 0:
                print('Worker %d assigned to task %d. Cost = %d' %
                        (smcf.tail(arc), smcf.head(arc), smcf.unit_cost(arc)))
else:
    print('There was an issue with the min cost flow input.')
    print('Status:', status)

Total cost = 265

Worker 1 assigned to task 8. Cost = 70
Worker 2 assigned to task 7. Cost = 55
Worker 3 assigned to task 6. Cost = 95
Worker 4 assigned to task 5. Cost = 45


## Assignment with teams of workers

This section presents a more general assignment problem. In this problem, six workers are divided into two teams. The problem is to assign four tasks to the workers so that the workload is equally balanced between the teams — that is, so each team performs two of the tasks.

In [64]:
# Declare solver
smcf = min_cost_flow.SimpleMinCostFlow()

# Define the directed graph for the flow.
team_a = [1, 3, 5]
team_b = [2, 4, 6]

start_nodes = ([0, 0] + [11, 11, 11] + [12, 12, 12] + [
    1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6
] + [7, 8, 9, 10])
end_nodes = ([11, 12] + team_a + team_b + [
    7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10, 7, 8,
    9, 10
] + [13, 13, 13, 13])
capacities = ([2, 2] + [1, 1, 1] + [1, 1, 1] + [
    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
] + [1, 1, 1, 1])
costs = ([0, 0] + [0, 0, 0] + [0, 0, 0] + [
    90, 76, 75, 70, 35, 85, 55, 65, 125, 95, 90, 105, 45, 110, 95, 115, 60,
    105, 80, 75, 45, 65, 110, 95
] + [0, 0, 0, 0])

source = 0
sink = 13
tasks = 4
# Define an array of supplies at each node.
supplies = [tasks, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -tasks]

The workers correspond to nodes 1 - 6. Team A consists of workers 1, 3, and 5, and team B consists of workers 2, 4, and 6. The tasks are numbered 7 - 10.

There are two new nodes, 11 and 12, between the source and workers. Node 11 is connected to the nodes for team A, and Node 12 is connected to the nodes for team B, with arcs of capacity 1. The graph below shows just the nodes and arcs from the source to the workers.

<p align = center>
    <img src = https://developers.google.com/static/optimization/images/flow/assignment_teams_workers.svg>
</p>

The key to balancing the workload is that the source 0 is connected to nodes 11 and 12 by arcs of capacity 2. This means that nodes 11 and 12 (and therefore teams A and B) can have a maximum flow of 2. As a result, each team can perform at most two of the tasks.

In [65]:
# Create constraints
# Add each arc
for i in range(0, len(start_nodes)):
    smcf.add_arc_with_capacity_and_unit_cost(start_nodes[i], end_nodes[i],
                                            capacities[i], costs[i])

# Add node supplies
for i in range(0, len(supplies)):
    smcf.set_node_supply(i, supplies[i])

# Invoke solver, find minimum cost flow between node 0 and node 10
status = smcf.solve()

# Output
if status == smcf.OPTIMAL:
    smcf.solve()
    print('Total cost =', smcf.optimal_cost(), '\n')
    for arc in range(smcf.num_arcs()):
        # Can ignore arcs leading out of source or intermediate, or into sink
        if (smcf.tail(arc) != source and smcf.tail(arc) != 11 and
            smcf.tail(arc) != 12 and smcf.head(arc) != sink):
            # Arcs in the solution will have a flow value of 1
            # Their start and end nodes give an assignment of worker to task
            if smcf.flow(arc) > 0:
                print(f'Worker {smcf.tail(arc)} assigned to task {smcf.head(arc)}. Cost = {smcf.unit_cost(arc)}')
else:
    print('There was an issue with the min cost flow input.')
    print('Status:', status)

Total cost = 250 

Worker 1 assigned to task 9. Cost = 75
Worker 2 assigned to task 7. Cost = 35
Worker 5 assigned to task 10. Cost = 75
Worker 6 assigned to task 8. Cost = 65
